# 1. Interactive Map with State Population (no Slider)

## Merge State Population with State Geometry

In [4]:
import geopandas as gpd

In [5]:
# Load file with state geometry
contiguous_usa = gpd.read_file('data/cb_2018_us_state_20m.shp')
contiguous_usa.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,24,01714934,0400000US24,24,MD,Maryland,00,25151100280,6979966958,"MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ..."
1,19,01779785,0400000US19,19,IA,Iowa,00,144661267977,1084180812,"POLYGON ((-96.62187 42.77925, -96.57794 42.827..."
2,10,01779781,0400000US10,10,DE,Delaware,00,5045925646,1399985648,"POLYGON ((-75.77379 39.72220, -75.75323 39.757..."
3,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702,"MULTIPOLYGON (((-82.86334 41.69369, -82.82572 ..."
4,42,01779798,0400000US42,42,PA,Pennsylvania,00,115884442321,3394589990,"POLYGON ((-80.51989 40.90666, -80.51964 40.987..."


In [6]:
type(contiguous_usa.iloc[0]['geometry'])

shapely.geometry.multipolygon.MultiPolygon

In [7]:
# Load file with 2018 state population
import pandas as pd
state_pop = pd.read_csv('data/state_pop_2018.csv')
state_pop.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,POPESTIMATE2018
0,10,0,0,0,United States,327167434
1,40,3,6,1,Alabama,4887871
2,40,4,9,2,Alaska,737438
3,40,4,8,4,Arizona,7171646
4,40,3,7,5,Arkansas,3013825


In [8]:
# Merge datasets
pop_states = contiguous_usa.merge(state_pop, left_on = 'NAME', right_on = 'NAME')

In [9]:
pop_states.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,SUMLEV,REGION,DIVISION,STATE,POPESTIMATE2018
0,24,01714934,0400000US24,24,MD,Maryland,00,25151100280,6979966958,"MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ...",40,3,5,24,6042718
1,19,01779785,0400000US19,19,IA,Iowa,00,144661267977,1084180812,"POLYGON ((-96.62187 42.77925, -96.57794 42.827...",40,2,4,19,3156145
2,10,01779781,0400000US10,10,DE,Delaware,00,5045925646,1399985648,"POLYGON ((-75.77379 39.72220, -75.75323 39.757...",40,3,5,10,967171
3,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702,"MULTIPOLYGON (((-82.86334 41.69369, -82.82572 ...",40,2,3,39,11689442
4,42,01779798,0400000US42,42,PA,Pennsylvania,00,115884442321,3394589990,"POLYGON ((-80.51989 40.90666, -80.51964 40.987...",40,1,2,42,12807060


In [10]:
# Drop Alaska and Hawaii
pop_states = pop_states.loc[~pop_states['NAME'].isin(['Alaska', 'Hawaii'])]

## Create Visualization with State Population

In [11]:
# Import packages for building map
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource, CustomJS,
                          CustomJSFilter, GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = pop_states.to_json())

In [12]:
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40000000)

# Define custom tick labels for color bar.
tick_labels = {'0': '0', '5000000': '5,000,000',
               '10000000':'10,000,000', '15000000':'15,000,000',
               '20000000':'20,000,000', '25000000':'25,000,000',
               '30000000':'30,000,000', '35000000':'35,000,000',
               '40000000':'40,000,000+'}

# Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p = figure(title = 'Lead Levels in Water Samples, 2018', plot_height = 600 ,
           plot_width = 950, toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'POPESTIMATE2018', 'transform' : color_mapper},
                   line_color = 'gray', line_width = 0.25, fill_alpha = 1)

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME'),('Population', '@POPESTIMATE2018')]))

# Specify layout
# p.add_layout(color_bar, 'below')

show(p)

# 2. Interactive Map of Water Data Overlaying State Population (with Slider)

## Load Water Data

In [15]:
sites_df = pd.read_csv('data/sites_2018.csv')
lead_samples = pd.read_csv('data/lead_samples_2018.csv')

//anaconda3/envs/geo-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (8,10,18,21,27,28,29,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
//anaconda3/envs/geo-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12,23,24,25,42,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
# Originally 43,010 sites
sites_subset = sites_df[['MonitoringLocationIdentifier', 'MonitoringLocationName',
                      'MonitoringLocationTypeName', 'LatitudeMeasure',
                      'LongitudeMeasure', 'StateCode', 'CountyCode']]
sites_subset.shape

(43010, 7)

In [17]:
# After dropping duplicates, 42,975 sites
sites_no_dup = sites_subset.drop_duplicates('MonitoringLocationIdentifier')
sites_no_dup.shape

(42975, 7)

In [18]:
# Originally 31,604 data points
lead_tests = lead_samples[['OrganizationIdentifier', 'OrganizationFormalName',
                           'ActivityIdentifier', 'ActivityTypeCode', 'ActivityMediaName', 'ActivityStartDate',
                           'MonitoringLocationIdentifier', 'ProjectIdentifier',
                           'SampleCollectionMethod/MethodIdentifier', 'SampleCollectionMethod/MethodName',
                           'CharacteristicName', 'ResultMeasureValue', 'ResultMeasure/MeasureUnitCode',
                           'ResultStatusIdentifier', 'ResultValueTypeName', 'PrecisionValue',
                           'ResultAnalyticalMethod/MethodIdentifier',
                           'ResultAnalyticalMethod/MethodIdentifierContext', 'ResultAnalyticalMethod/MethodName',
                           'DetectionQuantitationLimitTypeName', 'DetectionQuantitationLimitMeasure/MeasureValue',
                           'DetectionQuantitationLimitMeasure/MeasureUnitCode', 'ProviderName']]
lead_tests.shape

(31604, 23)

In [19]:
# After dropping n/a 16,637 data points
lead_tests_dropna = lead_tests[pd.notnull(lead_tests['ResultMeasureValue'])]
print(lead_tests_dropna.shape)

lead_levels = []
for x in list(lead_tests_dropna['ResultMeasureValue']):
    try:
        lead = float(x)
    except ValueError:
        lead = 'Not precise'
    lead_levels.append(lead)

lead_tests_dropna['LeadValue'] = lead_levels

# Drop where there is no lead_sites, 16,260 data points
lead_found = lead_tests_dropna[lead_tests_dropna['LeadValue'] != 'Not precise']

# Drop where lead was 0 ug/l, 14,934 data points
lead_found = lead_found[lead_found['LeadValue'] > 0]
print(lead_found.shape)

(16637, 23)
(14934, 24)


<ipython-input-19-481cfe72b6af>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lead_tests_dropna['LeadValue'] = lead_levels


In [20]:
# Subset where units are ug/l or mg/l, 14,328 data points
lead_per_l = lead_found[lead_found['ResultMeasure/MeasureUnitCode'].isin(['ug/l', 'mg/l'])]

lead_ug_per_l = []
units = list(lead_per_l['ResultMeasure/MeasureUnitCode'])
levels = list(lead_per_l['LeadValue'])
for i in range(0, len(lead_per_l)):
    if units[i] == 'mg/l':
        lead_ug_per_l.append(levels[i] * 1000)
    else:
        lead_ug_per_l.append(levels[i])

lead_per_l['LeadValue_ug_l'] = lead_ug_per_l
lead_per_l.shape

<ipython-input-20-db7454026fd2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lead_per_l['LeadValue_ug_l'] = lead_ug_per_l


(14328, 25)

## Merge Water Data with Geometry

In [24]:
# Merge lead levels with site location
lead_sites = lead_per_l.merge(sites_no_dup,
                              left_on = 'MonitoringLocationIdentifier',
                              right_on = 'MonitoringLocationIdentifier')

# Sort by testing date
lead_sites_sorted = lead_sites.sort_values(by = 'ActivityStartDate')
print(lead_sites_sorted.shape)
lead_sites_sorted.head()

(14328, 31)


,OrganizationIdentifier,OrganizationFormalName,ActivityIdentifier,ActivityTypeCode,ActivityMediaName,ActivityStartDate,MonitoringLocationIdentifier,ProjectIdentifier,SampleCollectionMethod/MethodIdentifier,SampleCollectionMethod/MethodName,...,DetectionQuantitationLimitMeasure/MeasureUnitCode,ProviderName,LeadValue,LeadValue_ug_l,MonitoringLocationName,MonitoringLocationTypeName,LatitudeMeasure,LongitudeMeasure,StateCode,CountyCode
733,USGS-CT,USGS Connecticut Water Science Center,nwisma.01.01800241,Sample-Routine,Water,2018-01-02,USGS-01127000,NaN,40,Multiple verticals,...,NaN,NWIS,0.214,0.214,"QUINEBAUG RIVER AT JEWETT CITY, CT",Stream,41.597492,-71.984094,9.0,11.0
6580,21IOWA_WQX,Iowa DNR Surface Water Monitoring Data,21IOWA_WQX-AMBIENT-SHL-606127-MET-SW,Sample-Routine,Water,2018-01-02,21IOWA_WQX-10580003,AMBIENT,UHL001,Standard UHL Sampling Procedure - Grab Samples,...,ug/l,STORET,0.08,0.080,Iowa River near Wapello,River/Stream,41.180331,-91.182147,19.0,115.0
6615,21IOWA_WQX,Iowa DNR Surface Water Monitoring Data,21IOWA_WQX-AMBIENT-SHL-606126-MET-SW,Sample-Routine,Water,2018-01-02,21IOWA_WQX-10700001,AMBIENT,UHL001,Standard UHL Sampling Procedure - Grab Samples,...,ug/l,STORET,0.1,0.100,Cedar River near Conesville,River/Stream,41.409359,-91.290171,19.0,139.0
6575,21IOWA_WQX,Iowa DNR Surface Water Monitoring Data,21IOWA_WQX-AMBIENT-SHL-606128-MET-SW,Sample-Routine,Water,2018-01-02,21IOWA_WQX-10580002,AMBIENT,UHL001,Standard UHL Sampling Procedure - Grab Samples,...,ug/l,STORET,0.05,0.050,Iowa River near Lone Tree,River/Stream,41.423865,-91.479090,19.0,103.0
6565,21IOWA_WQX,Iowa DNR Surface Water Monitoring Data,21IOWA_WQX-AMBIENT-SHL-606124-MET-SW,Sample-Routine,Water,2018-01-02,21IOWA_WQX-10560002,AMBIENT,UHL001,Standard UHL Sampling Procedure - Grab Samples,...,ug/l,STORET,0.09,0.090,Skunk River near Augusta,River/Stream,40.753400,-91.275590,19.0,111.0


In [23]:
# After dropping duplicates by date, 11,986 data points
lead_sites_dropdup = lead_sites_sorted.drop_duplicates(subset = ['MonitoringLocationIdentifier', 'ActivityStartDate'], keep = 'last').reset_index(drop = True)
print(lead_sites_dropdup.shape)

# Drop datapoints not in contiguous contiguous_usa, 10,078 data points
lead_sites_dropdup = lead_sites_dropdup[(lead_sites_dropdup['LongitudeMeasure'] <= -60) &
                                        (lead_sites_dropdup['LongitudeMeasure'] >= -130) &
                                        (lead_sites_dropdup['LatitudeMeasure'] <= 50) &
                                        (lead_sites_dropdup['LatitudeMeasure'] >= 20)]
print(lead_sites_dropdup.shape)

(11986, 31)
(10078, 31)


In [26]:
# Create Month column for plotting Slider
lead_sites_dropdup['Month'] = [int(x.split('-')[1]) for x in lead_sites_dropdup['ActivityStartDate']]

In [27]:
from shapely.geometry import Point

# Create shapely.Point objects based on longitude and latitude
geometry = []

for index, row in lead_sites_dropdup.iterrows():
    geometry.append(Point(row['LongitudeMeasure'], row['LatitudeMeasure']))

lead_sites_contig = lead_sites_dropdup.copy()
lead_sites_contig['geometry'] = geometry

# Save to dataframe
# lead_sites_contig.to_csv('lead_sites_contig_2018_per_l.csv')

## Final Version of Map with Lead Levels at Water Sites

In [28]:
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = pop_states.to_json())

# Read dataframe to geodataframe
lead_sites_crs = {'init': 'epsg:4326'}
lead_sites_geo = gpd.GeoDataFrame(lead_sites_contig,
                                  crs = lead_sites_crs,
                                  geometry = lead_sites_contig.geometry)

# Get x and y coordinates
lead_sites_geo['x'] = [geometry.x for geometry in lead_sites_geo['geometry']]
lead_sites_geo['y'] = [geometry.y for geometry in lead_sites_geo['geometry']]
p_df = lead_sites_geo.drop('geometry', axis = 1).copy()

sitesource = ColumnDataSource(p_df)

In [29]:
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40000000)

# Define custom tick labels for color bar.
tick_labels = {'0': '0', '5000000': '5,000,000',
               '10000000':'10,000,000', '15000000':'15,000,000',
               '20000000':'20,000,000', '25000000':'25,000,000',
               '30000000':'30,000,000', '35000000':'35,000,000',
               '40000000':'40,000,000+'}

# Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p = figure(title = 'Lead Levels in Water Samples, 2018', plot_height = 600 ,
           plot_width = 950, toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'POPESTIMATE2018', 'transform' : color_mapper},
                   line_color = 'gray', line_width = 0.25, fill_alpha = 1)

# Make a slider object to toggle the month shown
slider = Slider(title = 'Month',start = 1, end = 12, step = 1, value = 1)

# This callback triggers the filter when the slider changes
callback = CustomJS(args=dict(source=sitesource), code="""
    source.change.emit();
""")
slider.js_on_change('value', callback)

# Creates custom filter that selects the rows of the month based on the value in the slider
custom_filter = CustomJSFilter(args=dict(slider=slider, source=sitesource), code='''
var indices = [];
// iterate through rows of data source and see if each satisfies some constraint
for (var i = 0; i < source.get_length(); i++){
    if (source.data['Month'][i] == slider.value){
        indices.push(true);
    } else {
        indices.push(false);
    }
}
return indices;
''')

view = CDSView(source=sitesource, filters=[custom_filter])

# Plots the water sampling sites based on month in slider
sites = p.circle('x', 'y', source = sitesource, color = 'red',
                 size = 5, alpha = 0.3, view = view)

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME'),('Population', '@POPESTIMATE2018')]))
p.add_tools(HoverTool(renderers = [sites],
                      tooltips = [('Organization', '@OrganizationFormalName'),
                                  ('Location Type', '@MonitoringLocationTypeName'),
                                  ('Date', '@ActivityStartDate'),
                                  ('Lead (ug/l)', '@LeadValue_ug_l')]))
# Specify layout
p.add_layout(color_bar, 'below')

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p, widgetbox(slider))

show(layout)